In [7]:
!pip install -U scikit-learn imbalanced-learn matplotlib pandas numpy
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  

pd.set_option('display.max_columns', 100)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
df = pd.read_csv("C:\Users\HP\OneDrive\Desktop\Python\Day68,69\your_loan_dataset.csv")


target_col = "default"
df[target_col] = df[target_col].map({'No':0,'Yes':1}) if df[target_col].dtype=='O' else df[target_col].astype(int)

display(df.head())
print(df[target_col].value_counts(normalize=True).round(3))
print(df.isna().mean().sort_values(ascending=False).head(10)) 


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3310919775.py, line 1)

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.drop(target_col)
cat_cols = df.select_dtypes(exclude=[np.number]).columns
print("Numeric:", list(num_cols))
print("Categorical:", list(cat_cols))

from sklearn.compose import ColumnTransformer


num_cols = df.select_dtypes(include=["int64","float64"]).columns.drop("default")  
cat_cols = df.select_dtypes(include=["object"]).columns  


preprocess = ColumnTransformer(
    transformers=[("num", num_pre, num_cols),("cat", cat_pre, cat_cols)
    ],
    remainder="drop"   # drops unused columns like IDs
)



Numeric: ['income', 'loan_amount', 'age', 'dependents']
Categorical: ['customer_id', 'employment_type', 'credit_history', 'marital_status']


In [11]:
X = df.drop(columns=[target_col])
y = df[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
num_pre = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pre = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pre, num_cols),
        ("cat", cat_pre, cat_cols)
    ],
    remainder="drop"
)


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:

rfe_estimator = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)

rfe_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("smote", SMOTE(random_state=42)),
    ("rfe", RFE(estimator=rfe_estimator, n_features_to_select=25, step=0.1)),
    ("clf", LogisticRegression(max_iter=500))
])

rfe_pipe.fit(X_train, y_train)
rfe_preds = rfe_pipe.predict(X_test)
rfe_probs = rfe_pipe.predict_proba(X_test)[:,1]

print("=== RFE + Logistic Regression ===")
print(classification_report(y_test, rfe_preds, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, rfe_probs).round(3))


ohe2 = rfe_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["ohe"]
cat_feature_names2 = ohe2.get_feature_names_out(cat_cols)
all_feature_names2 = np.r_[num_feature_names, cat_feature_names2]

rfe_mask = rfe_pipe.named_steps["rfe"].get_support()
print("Selected features (RFE):", all_feature_names2[rfe_mask].tolist())


NameError: name 'preprocess' is not defined